In [ ]:
# download ESACCI L4 SSS 
#! PYTHONPATH=../../pycommon ./get_l4_sss_esacci.py \
#         --start_date 20180101 --end_date 20191231 \
#         --log 

In [ ]:
# Updated configurations under namelist "regrid": L4 -> WOA18
! cat config/config.merge_sal_esacci_woa18.yaml

In [ ]:
# generate interpolating weights: L4 -> WOA18
! time PYTHONPATH=../../pycommon ./gen_regridder_esacci_to_woa18.py config/config.merge_sal_esacci_woa18.yaml

In [ ]:
# Updated configurations under namelist "regrid": WOA18 -> MOM 
! cat config/config.merge_sal_esacci_woa18.yaml

In [ ]:
# generate interpolating weights: WOA18 -> MOM 
! time PYTHONPATH=../../pycommon ../woa/gen_regridder_woa18_to_mom.py config/config.merge_sal_esacci_woa18.yaml

In [ ]:
# generate 12-month salinity files from WOA18
! PYTHONPATH=../../pycommon ../woa/fill_sss_woa18_12months_daily.py ../../test_data/woa18 \
 --file_name_template "woa18_decav_s{:02d}_04.nc" \
 --start_month 1 --end_month 12 \
 --depth 10 \
 --woa_var s_an \
 --woa_var_renamed SALT 

In [ ]:
# create daily merged L4&WOA SSS on WOA18 grids

! time PYTHONPATH=../../pycommon ./merge_sal_esacci_woa18_daily.py \
        --config_path config/config.merge_sal_esacci_woa18.yaml \
        --woa18_filled_path ../woa/sss_woa18_10m/filled_sss_s01_e12.nc \
        --l4_sss_path TEST_DATA/l4_sss_esacci/2019/201904/20190430/ESACCI-SEASURFACESALINITY-L4-SSS-MERGED_OI_7DAY_RUNNINGMEAN_DAILY_25km-20190430-fv3.21.nc \
        --radius 10 \
        --output_file_path NEW_test_l4_1day.nc 

In [ ]:
# generate daily SSS restore file in paralell

#!cat create_Srestore_batch.bsh
!bash create_Srestore_batch.bsh

In [23]:
!ncrcat -h TEST_DATA/remapped_l4_sss_v2/NEW2_sss_remapped_201501*.nc filled_remapped_cda_ESACCI_SSS_v2.nc